Caixeiro viajante
=================



## Introdução



Até agora, sempre que nós aplicamos os operadores de `cruzamento` ou de `mutação` nós não nos preocupamos se o indivíduo gerado por estes processo era um `indivíduo válido`. Um indivíduo válido é aquele que representa uma solução possível e bem formulada para o problema em questão.

Por exemplo, no problema das caixas binárias, [1, 0, 0, 1] é um indivíduo válido para o caso de termos 4 caixas. Um exemplo de `indivíduo inválido` para este mesmo problema seria [1, 0, 0, a], pois um dos genes está assumindo um valor fora do domínio. Outro exemplo de indivíduo inválido poderia ser [1, 1, 0], pois é um indivíduo com apenas 3 genes, sendo que o esperado eram 4 genes.

Neste experimento nós veremos estratégias para evitar que indivíduos inválidos sejam obtidos quando usamos os operadores de cruzamento e de mutação. No notebook seguinte veremos como aplicar uma penalidade para indivíduos inválidos que forem gerados durante uma busca genética com restrições.



## Objetivo



Encontrar uma solução para o problema do caixeiro viajante. Considere que ele irá visitar 5 cidades, pode iniciar sua viagem por qualquer uma destas cidades e deve retornar à cidade de início. Durante seu trajeto, não pode visitar a mesma cidade duas vezes (única exceção é a cidade inicial).



## Descrição do problema



O problema consiste em descobrir a rota de menor distância entre $n$ pontos no plano cartesiano (ou seja, $n$ pontos com coordenadas $(x,y)$). A rota pode se iniciar em qualquer um dos pontos disponíveis e deve terminar no ponto inicial, visitando todos os demais pontos apenas uma vez. Considere que a rota entre um ponto e outro é a linha reta que liga os dois pontos.

## Importações



In [1]:
import random
from itertools import permutations

from funcoes import cria_cidades

from funcoes import populacao_inicial_cv as cria_populacao_inicial
from funcoes import funcao_objetivo_pop_cv
from funcoes import selecao_torneio_min # esse já temos!
from funcoes import cruzamento_ordenado as funcao_cruzamento
from funcoes import mutacao_de_troca as funcao_mutacao

from funcoes import funcao_objetivo_cv as funcao_objetivo_individuo

from time import perf_counter as pf

## Códigos e discussão



In [2]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 50
NUM_GERACOES = 1000
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05
NUM_COMBATENTES_NO_TORNEIO = 3

# relacionadas ao problema a ser resolvido
NUMERO_DE_CIDADES = 10
CIDADES = cria_cidades(NUMERO_DE_CIDADES)

In [3]:
# Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_cv(populacao, CIDADES)

def funcao_selecao(populacao, fitness):
    return selecao_torneio_min(populacao, fitness, NUM_COMBATENTES_NO_TORNEIO)

In [4]:
# Busca por algoritmo genético

start_algoritmo = pf()

populacao = cria_populacao_inicial(TAMANHO_POP, CIDADES)

melhor_fitness_ja_visto = float("inf")  # é assim que escrevemos infinito em python

for n in range(NUM_GERACOES):
    
    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)
    
    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]
    
    contador = 0
    
    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2
        
        contador = contador + 2   
        
    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)            
            
    # melhor individuo já visto até agora
    fitness = funcao_objetivo_pop(populacao)
    menor_fitness = min(fitness)
    if menor_fitness < melhor_fitness_ja_visto:        
        posicao = fitness.index(menor_fitness)
        melhor_individuo_ja_visto = populacao[posicao]
        melhor_fitness_ja_visto = menor_fitness    
    
fim_algoritmo = pf()

In [5]:
# Busca testando todas as permutações

melhor_fitness_ever = float("inf")

inicio_all = pf()

# testando todas as permutações possíveis
for caminho in permutations(list(CIDADES.keys())):
    caminho = list(caminho)
    distancia = funcao_objetivo_individuo(caminho, CIDADES)
    if distancia < melhor_fitness_ever:
        melhor_fitness_ever = distancia
        melhor_resposta_ever = caminho

fim_all = pf()

In [6]:
# Checando os resultados

print()
print("Melhor individuo obtido por algoritmos genéticos:")
print(melhor_individuo_ja_visto, "com distância:", melhor_fitness_ja_visto, f'em {fim_algoritmo-start_algoritmo} segundos')

print()
print("Melhor individuo obtido por busca exaustiva:")
print(melhor_resposta_ever, "com distância:", melhor_fitness_ever, f'em {fim_all-inicio_all} segundos')


Melhor individuo obtido por algoritmos genéticos:
['Cidade 8', 'Cidade 0', 'Cidade 5', 'Cidade 4', 'Cidade 2', 'Cidade 7', 'Cidade 3', 'Cidade 6', 'Cidade 1', 'Cidade 9'] com distância: 2.300715601402959 em 0.9403050999999998 segundos

Melhor individuo obtido por busca exaustiva:
['Cidade 0', 'Cidade 5', 'Cidade 4', 'Cidade 7', 'Cidade 9', 'Cidade 3', 'Cidade 6', 'Cidade 1', 'Cidade 2', 'Cidade 8'] com distância: 2.300715601402959 em 53.073686599999995 segundos


## Conclusão

O objetivo do notebook era de implementar um algoritmo genético que fosse capaz de chegar na solução do problema do caixeiro viajante para 5 cidades (colocado em 10 no notebook para comparação com o método exaustivo de checagem), este objetivo foi alcançado, como é possível ver pela comparação entre o algoritmo implementado e o método exaustivo. Os dois métodos retornaram o mesmo resultado (por mais que transladado), demosntrando que o resultado obtido através do algorítmo é realmente a melhor solução para o problema do caixeiro viajante para esta situação, indicando o sucesso em sua implementação.

Para que este fosse implementado de maneira correta algumas mudanças tiveram que ser feitas, como a mutação que agora deve mudar apenas a ordem das cidades dentro de um individuo e o cruzamento que deve manter a integridade dos dois individuos resultantes do cruzamento. Outra diferença é a ausência de uma função para o gêne, já que temos que considerar a necessidade da existência de todas as cidades dentro do indivíduo sem repetição.

De modo geral o algoritmo teve um resultado altamente eficiente, principalmente quando o comparamos com o método de "força bruta". Enquanto o algoritmo resolveu o problema com apenas 1000 geração em menos de 1 segundo (aproximandamente 0.94s), o método de "força bruta" teve uma eficiência muito pior, precisando de 53s. Caso o problema seja aumentado para mais cidades, o segundo método irá piorar com uma decadência de escala fatorial, demosntrando a enorme vantagem do código de algoritmo genético, que, apesar de não garantir a melhor resposta, este irá retornar a melhor solução encontrada em tempos extremamente menores 

## Playground



In [7]:
pai = [1,2,3,4,5]
mae = [5,4,3,2,1]

corte1 = random.randint(0, len(pai) - 2)
corte2 = random.randint(corte1 + 1, len(pai) - 1)

filho1 = pai[corte1:corte2]
filho2 = mae[corte1:corte2]

for a in [i for i in mae if i not in filho1]:
    filho1.append(a)
filho1

[1, 2, 5, 4, 3]

In [8]:
individuo = [1,2,3,4,5]

indices = list(range(len(individuo)))
indice1, indice2 = random.sample(indices, k=2)

print(individuo)

individuo[indice1], individuo[indice2] = individuo[indice2], individuo[indice1]

print(individuo)

[1, 2, 3, 4, 5]
[1, 2, 3, 5, 4]


In [9]:
from funcoes import distancia_entre_dois_pontos
from collections import deque

cities = cria_cidades(5)
print(cities)

nomes = list(cities.keys())
random.shuffle(nomes)
print(nomes)

nomes_copy = deque(nomes)
nomes_copy.rotate(-1)
print(nomes_copy)

{'Cidade 0': (0.2324112350458568, 0.9414032935701182), 'Cidade 1': (0.3765692324088985, 0.09988466274101115), 'Cidade 2': (0.06145705427557968, 0.2617111824102414), 'Cidade 3': (0.01901889443802296, 0.6620855376921005), 'Cidade 4': (0.002852464995091508, 0.4135210927963593)}
['Cidade 4', 'Cidade 3', 'Cidade 2', 'Cidade 1', 'Cidade 0']
deque(['Cidade 3', 'Cidade 2', 'Cidade 1', 'Cidade 0', 'Cidade 4'])
